In [4]:
import pandas as pd
import numpy as np
import subprocess
import json
import os
from dotenv import load_dotenv
from constants import *
from web3 import Web3
from bit import wif_to_key, PrivateKeyTestnet
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from eth_account import Account
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy



load_dotenv()

True

In [5]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer =0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [6]:
mnemonic = os.getenv('MNEMONIC')

In [7]:
def derive_wallets (mnemonic, coin, number):
    command = f"./derive -g --mnemonic ='{mnemonic}' --coin='{coin}' --numderive='{number}' --cols=index,path,address,privkey,pubkey,pubkeyhash,xprv,xpub --format=json"
    #suprocess.PIPE redirects the command output so that we can use it. 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) =p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys


In [8]:
class coin:
    BTC = 'btc'
    ETH = 'eth'
    BTCTEST = 'btc-test'

In [75]:
derive_wallets(mnemonic, ETH, 3)

[{'index': 0,
  'path': "m/44'/60'/0'/0/0",
  'address': '0xdeab62bC0fF25Af08B596DAA032c8a4838dc42c9',
  'privkey': '0x1ca051ae151b0929b5342d38628033ae1b36a8140ebbcbb7b856206c6c217f3b',
  'pubkey': '0317c874eba5618e7e53b9b31a03d5a334f50cb8507f2762d9467bf8627bf07cd5',
  'pubkeyhash': '44d218f0d353366c40e0743c32f3c97329852541',
  'xprv': 'xprvA2naic8XCtoQrNqQrCRb1qynRcS4gqUmavNGgVH7d1YATXH5VYaP1pZMMN37MQ8bVpz1eLcTNC1bhPH3zkxGqjtWCt1iJ4EnwMFVZKa4jXd',
  'xpub': 'xpub6Fmw87fR3GMi4rusxDxbNyvWyeGZ6JCcx9HsUsgjBM59LKcE35tdZcsqCeyVFttjBv2GhYuLtunhUPbC9fkAWw7y8rxHKJTPEDoqWgmdogd'},
 {'index': 1,
  'path': "m/44'/60'/0'/0/1",
  'address': '0x992D3236D69691549339dEf73141072714e77eB4',
  'privkey': '0xafc39ebebf833abf3f16affc199015933edbff033c846be44895a414057f8003',
  'pubkey': '03b78e83277d729a9af6239776dfb0ebce9f7e8fb39fb5bd326c8ed60eeb96f970',
  'pubkeyhash': 'b94ff208e1eb5ffd37fc6ea6b6f2d595689935fd',
  'xprv': 'xprvA2naic8XCtoQupQqFSZ7qxdxeSPxfymSUX8ru4cGCsMvCJkJX3eEAHiUz83T41KaFRxyUrs8DKB2nq

In [65]:
def priv_key_to_account (coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)


In [66]:
acc = priv_key_to_account(BTCTEST, derive_wallets(mnemonic, BTCTEST, 5)[0]['privkey'])

In [67]:
print (acc)

<PrivateKeyTestnet: muntSzUFsYrksr3aDuWRUSSYumm5UxkdT4>


In [73]:
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": to, "value": amount}
        )
        return {
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])


In [74]:
create_tx(BTCTEST, acc, 'mzqkXKRfqYgcXWYWkNfjNV3h9e5cGhhnwP', 0.001)

'{"unspents":[{"amount":1908780,"confirmations":6,"script":"76a9149c9534ba0b42be5721f738e7b26b1982a2e53b7c88ac","txid":"21376bd607b3297a5c993e645f3f3cccc066b6f8a7bce8559c1ffc906938a80b","txindex":1,"type":"p2pkh","vsize":148,"segwit":false}],"outputs":[["mzqkXKRfqYgcXWYWkNfjNV3h9e5cGhhnwP",100000],["muntSzUFsYrksr3aDuWRUSSYumm5UxkdT4",1783468]]}'

In [50]:
def send_tx(coin,account, recipient, amount):
    tx = create_tx(coin,account,recipient,amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [51]:
send_tx(ETH, acc, '0xAaB1FE67ad236DDA66038Aa46710A9e9De81AC0d', 10)

ValueError: {'message': "sender doesn't have enough funds to send tx. The upfront cost is: 420000000000010 and the sender's account only has: 0", 'code': -32000, 'data': {'stack': "Error: sender doesn't have enough funds to send tx. The upfront cost is: 420000000000010 and the sender's account only has: 0\n    at VM.<anonymous> (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache-core/node_modules/ethereumjs-vm/lib/runTx.ts:114:11)\n    at step (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache-core/node_modules/ethereumjs-vm/dist/runTx.js:32:23)\n    at Object.next (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache-core/node_modules/ethereumjs-vm/dist/runTx.js:13:53)\n    at fulfilled (/Applications/Ganache.app/Contents/Resources/static/node/node_modules/ganache-core/node_modules/ethereumjs-vm/dist/runTx.js:4:58)\n    at runMicrotasks (<anonymous>)\n    at processTicksAndRejections (internal/process/task